<a href="https://colab.research.google.com/github/Pakhi27/Hope-Dataset-Audio-ALL-Classification-Algorithms/blob/main/Hope_Dataset_Audio_ALL_Classification_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install gtts moviepy


In [2]:
import pandas as pd
import pyttsx3

# Load your dataset
df = pd.read_excel('/content/Emotion_DS-3E.xlsx')

# Check DataFrame shape
print(f"DataFrame shape: {df.shape}")

# Check if 'Utterance' column exists
if 'Utterance' not in df.columns:
    raise ValueError("The 'Utterance' column is missing from the dataset.")

# Initialize the TTS engine
engine = pyttsx3.init()

# Create a new column for audio file names
df['AudioFile'] = ""

# Function to create audio for each utterance
def create_audio(utterance, index):
    try:
        # Save the audio to a file
        audio_file = f"audio_{index}.mp3"  # Unique audio file for each utterance
        engine.save_to_file(utterance, audio_file)
        engine.runAndWait()

        # Save the audio file name to the DataFrame
        df.at[index, 'AudioFile'] = audio_file  # Update the DataFrame with the audio file name

        print(f"Audio saved as {audio_file}")
    except Exception as e:
        print(f"Error processing index {index}: {e}")

# Iterate over the utterances in the DataFrame using df.iterrows()
for idx, row in df.iterrows():
    utterance = row['Utterance']  # Get the utterance from the current row
    print(f"Processing index {idx}: {utterance}")  # Log the index and utterance
    create_audio(utterance, idx)  # Create audio for the utterance

# Save the updated DataFrame with audio file names to a new CSV
df.to_csv('updated_dataset_with_audio_files.csv', index=False)

print("Audio file names have been added to the DataFrame and saved to 'updated_dataset_with_audio_files.csv'.")


Streaming output truncated to the last 5000 lines.
Audio saved as audio_2747.mp3
Processing index 2748: nan
Audio saved as audio_2748.mp3
Processing index 2749: nan
Audio saved as audio_2749.mp3
Processing index 2750: nan
Audio saved as audio_2750.mp3
Processing index 2751: nan
Audio saved as audio_2751.mp3
Processing index 2752: nan
Audio saved as audio_2752.mp3
Processing index 2753: nan
Audio saved as audio_2753.mp3
Processing index 2754: nan
Audio saved as audio_2754.mp3
Processing index 2755: nan
Audio saved as audio_2755.mp3
Processing index 2756: nan
Audio saved as audio_2756.mp3
Processing index 2757: nan
Audio saved as audio_2757.mp3
Processing index 2758: nan
Audio saved as audio_2758.mp3
Processing index 2759: nan
Audio saved as audio_2759.mp3
Processing index 2760: nan
Audio saved as audio_2760.mp3
Processing index 2761: nan
Audio saved as audio_2761.mp3
Processing index 2762: nan
Audio saved as audio_2762.mp3
Processing index 2763: nan
Audio saved as audio_2763.mp3
Process

In [6]:
# Print the absolute path of the saved CSV file
csv_absolute_path = os.path.abspath('updated_dataset_with_audio_files.csv')
print("Absolute path of the saved CSV file:", csv_absolute_path)

Absolute path of the saved CSV file: /content/updated_dataset_with_audio_files.csv


In [8]:
from google.colab import files


In [9]:
# Download the CSV file
files.download('updated_dataset_with_audio_files.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import os

# Existing code...

# Save the updated DataFrame with audio file names to a new CSV
df.to_csv('updated_dataset_with_audio_files.csv', index=False)

# Print the current working directory
print("Audio file names have been added to the DataFrame and saved to 'updated_dataset_with_audio_files.csv'.")
print("Current working directory:", os.getcwd())


Audio file names have been added to the DataFrame and saved to 'updated_dataset_with_audio_files.csv'.
Current working directory: /content


In [11]:
import os
import numpy as np
import pandas as pd
import librosa

# Load your updated dataset
df = pd.read_csv('updated_dataset_with_audio_files.csv')

# Function to extract MFCC features
def extract_mfcc(file_path, default_n_fft=512):  # Set a default value for n_fft
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)  # Load audio file

        # Determine n_fft based on the length of the audio signal
        n_fft = min(default_n_fft, len(audio))  # Ensure n_fft is not greater than audio length

        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13, n_fft=n_fft)  # Extract MFCCs
        mfccs_mean = np.mean(mfccs.T, axis=0)  # Mean of MFCCs over time
        return mfccs_mean
    except Exception as e:
        print(f"Error extracting MFCC from {file_path}: {e}")
        return None

# Prepare a new DataFrame to hold the features and labels
features = []
labels = []  # You should have your corresponding emotion labels in the DataFrame

# Iterate through each audio file in the DataFrame
for index, row in df.iterrows():
    audio_file = row['AudioFile']
    emotion_label = row['Emotion']  # Adjust according to your DataFrame structure

    mfccs = extract_mfcc(audio_file)
    if mfccs is not None:
        features.append(mfccs)
        labels.append(emotion_label)  # Assuming you have an 'Emotion' column

# Convert to NumPy arrays
X = np.array(features)
y = np.array(labels)

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")


/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Features shape: (5247, 13)
Labels shape: (5247,)


Support Vector Machine(SVM)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.67      0.01      0.02       212
           0       0.79      1.00      0.88       827
           1       0.00      0.00      0.00        11

    accuracy                           0.79      1050
   macro avg       0.49      0.34      0.30      1050
weighted avg       0.76      0.79      0.70      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Prediction with new sample

In [15]:
# Function to predict emotion from a new audio file
def predict_emotion(file_path):
    mfccs = extract_mfcc(file_path)
    if mfccs is not None:
        mfccs = mfccs.reshape(1, -1)  # Reshape for prediction
        emotion = model.predict(mfccs)
        return emotion[0]  # Return the predicted emotion
    return None

# Example prediction
new_audio_file = "C:\\Users\\singh\\Downloads\\audio_1.mp3"
predicted_emotion = predict_emotion(new_audio_file)
print(f"Predicted Emotion: {predicted_emotion}")


Error extracting MFCC from C:\Users\singh\Downloads\audio_1.mp3: [Errno 2] No such file or directory: 'C:\\Users\\singh\\Downloads\\audio_1.mp3'
Predicted Emotion: None


<ipython-input-11-a1cf540ae99c>:12: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)  # Load audio file


In [36]:
import pandas as pd
import random

# Load your updated dataset
df = pd.read_csv('updated_dataset_with_audio_files.csv')

# Select a random audio file from the DataFrame
random_index = random.choice(df.index)  # Choose a random index
random_audio_file = df.at[random_index, 'AudioFile']  # Get the corresponding audio file path

# Function to predict emotion from a new audio file
def predict_emotion(file_path):
    mfccs = extract_mfcc(file_path)
    if mfccs is not None:
        mfccs = mfccs.reshape(1, -1)  # Reshape for prediction
        emotion = model.predict(mfccs)
        return emotion[0]  # Return the predicted emotion
    return None

# Predict emotion for the randomly selected audio file
predicted_emotion = predict_emotion(random_audio_file)

print(f"Randomly selected Audio File: {random_audio_file}")
print(f"Predicted Emotion: {predicted_emotion}")


Randomly selected Audio File: audio_4805.mp3
Predicted Emotion: 0


In [35]:
df['Emotion'].value_counts()

,Emotion
0,4098
-1,1070
1,79


Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       212
           0       0.79      1.00      0.88       827
           1       0.00      0.00      0.00        11

    accuracy                           0.79      1050
   macro avg       0.26      0.33      0.29      1050
weighted avg       0.62      0.79      0.69      1050



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest Classification Report:
              precision    recall  f1-score   support

          -1       0.57      0.06      0.11       212
           0       0.80      0.99      0.88       827
           1       0.00      0.00      0.00        11

    accuracy                           0.79      1050
   macro avg       0.45      0.35      0.33      1050
weighted avg       0.74      0.79      0.72      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gradient Boosting (XGBoost)

In [21]:
# Check unique values in y
unique_labels = np.unique(y)
print(f"Unique labels in the dataset: {unique_labels}")


Unique labels in the dataset: [-1  0  1]


In [22]:
# Map your emotion labels to a range starting from 0
label_mapping = {value: idx for idx, value in enumerate(sorted(set(y)))}
y_mapped = np.vectorize(label_mapping.get)(y)

# Check the unique values of the new labels
print(f"Mapped labels: {np.unique(y_mapped)}")


Mapped labels: [0 1 2]


In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets using the mapped labels
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:47:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.07      0.11       212
           1       0.79      0.98      0.88       827
           2       0.00      0.00      0.00        11

    accuracy                           0.78      1050
   macro avg       0.41      0.35      0.33      1050
weighted avg       0.71      0.78      0.71      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KNN

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predict on the test set
y_pred_knn = knn_model.predict(X_test)

# Evaluate the model
print("K-Nearest Neighbors Classification Report:")
print(classification_report(y_test, y_pred_knn))


K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

          -1       0.34      0.10      0.16       212
           0       0.80      0.95      0.87       827
           1       0.00      0.00      0.00        11

    accuracy                           0.77      1050
   macro avg       0.38      0.35      0.34      1050
weighted avg       0.70      0.77      0.71      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_model.predict(X_test)

# Evaluate the model
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))


Decision Tree Classification Report:
              precision    recall  f1-score   support

          -1       0.29      0.11      0.16       212
           0       0.80      0.92      0.85       827
           1       0.00      0.00      0.00        11

    accuracy                           0.75      1050
   macro avg       0.36      0.35      0.34      1050
weighted avg       0.68      0.75      0.71      1050

